# Ionization states

The ionization state distribution for an element refers to the fractions of that element at each ionic level.  For example, the charge state of helium might be 10% He$^{0+}$, 70% He$^{1+}$, and 20% He$^{2+}$. 

In [ ]:
from plasmapy.particles import IonizationState, IonizationStateCollection

## The ionization state of a single element

Let's store this information in an `IonizationState` object, and specify the number density of the element too.

In [ ]:
He_states = IonizationState("He-4", [0.1, 0.7, 0.2], n_elem=1e7 * u.cm**-3)

The ionization state distribution is stored in the `ionic_fractions` attribute.

In [ ]:
He_states.ionic_fractions

We can get the symbols for each ionic level too.

In [ ]:
He_states.ionic_symbols

Because we provided the number density of the element as a whole, we can get back the number density of each ionic level.

In [ ]:
He_states.number_densities

We can also get the electron number density required to balance the positive charges for ions of this element.

In [ ]:
He_states.n_e

We can provide an `IonizationState` with a charge number as an index to get an `IonicLevel` object that contains most of these attributes, but for a single ionic level (like He$^{1+}$).  This capability is useful if we wish to iterate over the ions of an element.

In [ ]:
for Z in range(3):
    print(He_states[Z])

We can get information about the average charge state via `Z_mean`, `Z_most_abundant`, and `Z_rms`.

In [ ]:
He_states.Z_mean

In [ ]:
He_states.Z_most_abundant

In [ ]:
He_states.Z_rms

We can calculate the properties of the average ionic level.

In [ ]:
He_states.average_ion()

We can use the `summarize()` method to get information about the ionization state.

In [ ]:
He_states.summarize()

## Ionization states of multiple elements

[Gilbert et al. (2012)]: https://doi.org/10.1088/0004-637X/751/1/20

Now let's look at some actual average ionization state data for filament material in an ICME observed by the *Advanced Composition Explorer* (*ACE*) near 1 AU.  The data were estimated from Figure 4 in [Gilbert et al. (2012)].  These data are noteworthy because there is information from very low charge states to very high charge states.

In [ ]:
number_densities = {
    "C": [0, 5.7e-7, 4.3e-5, 3.6e-6, 2.35e-6, 1e-6, 1.29e-6] * u.cm**-3,
    "O": [0, 1.2e-7, 2.2e-4, 7.8e-6, 8.8e-7, 1e-6, 4e-6, 1.3e-6, 1.2e-7] * u.cm**-3,
    "Fe": [
        0,
        0,
        1.4e-8,
        1.1e-7,
        2.5e-7,
        2.2e-7,
        1.4e-7,
        1.2e-7,
        2.1e-7,
        2.1e-7,
        1.6e-7,
        8e-8,
        6.3e-8,
        4.2e-8,
        2.5e-8,
        2.3e-8,
        1.5e-8,
        3.1e-8,
        6.1e-9,
        2.3e-9,
        5.3e-10,
        2.3e-10,
        0,
        0,
        0,
        0,
        0,
    ]
    * u.cm**-3,
}

Now let's use this information as an input for `IonizationStateCollection`: a data structure for the ionization states of multiple elements.

In [ ]:
states = IonizationStateCollection(number_densities)

We can index this to get an `IonizationState` for one of the elements.

In [ ]:
states["C"]

We can get the relative abundances of each of the elements.

In [ ]:
states.abundances

In [ ]:
states.log_abundances

We can get the number densities as a `dict` (like what we provided) and the electron number density (assuming quasineutrality, but only for the elements contained in the data structure).

In [ ]:
states.number_densities

In [ ]:
states.n_e

We can summarize this information too, but let's specify the minimum ionic fraction to print.

In [ ]:
states.summarize(minimum_ionic_fraction=0.1)

Now let's make some plots!